In [1]:
import pandas as pd
import re
import hashlib

In [2]:
import os
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [3]:
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')

'SMARTROLL.settings'

In [4]:
django.setup()

In [5]:
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student
from django.contrib.auth import get_user_model

In [6]:
User = get_user_model()

In [7]:
college_pattern = r"(?P<college_name>[\w\s\.]+)-(?P<college_code>\d+)"
term_pattern = r"(?P<start_year>\d{4})-(?P<end_year>\d{4})"
branch_pattern = r"(?P<branch_name>[\w\s]+)-(?P<branch_code>\d{2})"
college_info = {}
term_info = {}
branch_info = {}

In [8]:
df = pd.read_csv('smartroll.csv')
header_rows = df.iloc[:3].reset_index(drop=True)
for i, value in enumerate(header_rows.iloc[:, 0]):
    if i == 0:        
        match = re.search(college_pattern, value)
        if match:
            college_info["college_name"] = match.group("college_name").strip()
            college_info["college_code"] = match.group("college_code")
    
    elif i == 1:        
        match = re.search(term_pattern, value)
        if match:
            term_info["start_year"] = match.group("start_year")
            term_info["end_year"] = match.group("end_year")
    
    elif i == 2:        
        match = re.search(branch_pattern, value)
        if match:
            branch_info["branch_name"] = match.group("branch_name").strip()
            branch_info["branch_code"] = match.group("branch_code")

# Get the college,Term, Branch Objects
college_obj = College.objects.filter(code=college_info['college_code']).first()
term_obj = Term.objects.filter(start_year=term_info['start_year'],end_year=term_info['end_year']).first()
branch_obj = Branch.objects.filter(branch_code=branch_info['branch_code']).first()
print(college_obj)
print(term_obj)
print(branch_obj)

L.D. College of Engineering
Term - 2024 | 2025
Computer Engineering | Term - 2024 | 2025


In [9]:
df.columns = df.iloc[3]
df = df[4:].reset_index(drop=True)  
faculties = list(df.columns[2:])

In [10]:
BE_PATTERN = re.compile(r"^(BE)-(\d+)-([A-Z])(?:_([A-Z]\d))?-([A-Z]+(?:-\w+)?)-(\w+)$")
ME_PATTERN = re.compile(r"^(ME)-(\d+)-([A-Z]+(?:-\w+)?)-(\d+)$")
TIME_PATTERN = re.compile(r"^(\d{2}:\d{2})-(\d{2}:\d{2})$")

In [11]:
def parse_be_me_string(string):
    # Try BE pattern first
    be_match = BE_PATTERN.match(string)
    if be_match:
        course, sem, div, batch, sub, classroom = be_match.groups()
        return {
            "stream": course,
            "sem": sem,
            "div": div,
            "batch": batch if batch else None,  # If no batch, set to None
            "sub": sub,
            "classroom": classroom
        }
    
    # Try ME pattern
    me_match = ME_PATTERN.match(string)
    if me_match:
        course, sem, sub, classroom = me_match.groups()
        return {
            "stream": course,
            "sem": sem,
            "sub": sub,
            "classroom": classroom
        }
    
    # If no pattern matches
    return None

def parse_time_string(time_string):
    time_match = TIME_PATTERN.match(time_string)
    if time_match:
        start_time, end_time = time_match.groups()
        return {"start_time": start_time, "end_time": end_time}
    return None

def hash_string(input_string):
    # Convert the string to bytes
    byte_string = input_string.encode()
    # Create a SHA-256 hash of the byte string
    hash_object = hashlib.sha256(byte_string)
    # Return the hex representation of the hash
    return hash_object.hexdigest()

In [35]:
for day, day_data in df.groupby('DAY'):
    print(f"Schedule for {day}:")
    rows = list(day_data.iterrows())
    i = 0
    while i < len(rows):
        index, row = rows[i]        
        for faculty in faculties:                                  
            time_info = parse_time_string(row['TIME'])
            if time_info:
                info = parse_be_me_string(row[faculty]) if pd.notna(row[faculty]) else None                
                if pd.notna(row[faculty]):
                    current_lecture_hash = hash_string(row[faculty])
                    # Also check if next consecutive lecture is not the same as current
                    if i < len(rows) - 1: 
                        next_row = rows[i+1][1]
                        next_time_info = parse_time_string(next_row['TIME'])
                        next_info = parse_be_me_string(next_row[faculty]) if pd.notna(next_row[faculty]) else None
                        if pd.notna(next_row[faculty]):
                            next_lecture_hash = hash_string(next_row[faculty])
                            if next_lecture_hash == current_lecture_hash:
                                time_info['end_time'] = next_time_info['end_time']
                                rows[i+1][1][faculty] = None          
                # Now we can store the lecture data
                if info:
                    # Get the stream first
                    stream_obj = branch_obj.stream_set.filter(title=info['stream']).first()
                    if stream_obj:
                        # Get the semester                        
                        semester_obj = stream_obj.semester_set.filter(no=info['sem']).first()
                        if semester_obj:
                            # get the division 
                            # For ME there will be only 1 division per semester                                                        
                            if info['stream'] == 'ME':
                                division_obj = semester_obj.division_set.first()
                                #  Get the first batch object of the division
                                batch_obj = division_obj.batch_set.first()                                
                            elif info['stream'] == 'BE':
                                division_obj = semester_obj.division_set.filter(division_name=info['div']).first()
                                # Get the batch
                                # We have to check if the lecture is for the whole class or single batch
                                if info['batch']:
                                    print("batch")
                                else:
                                    print('All Batches')
                                # batch_obj = division_obj.batch_set.filter(batch_name=)
                                                          
        i += 1
    print()    

Schedule for FRIDAY:

Schedule for MONDAY:

Schedule for SATURDAY:

Schedule for THURSDAY:

Schedule for TUESDAY:
Batch - A1 | Division - A | Semester - 3 | Stream - ME | Computer Engineering | Term - 2024 | 2025
Batch - A1 | Division - A | Semester - 1 | Stream - ME | Computer Engineering | Term - 2024 | 2025

Schedule for WEDNESDAY:
Batch - A1 | Division - A | Semester - 3 | Stream - ME | Computer Engineering | Term - 2024 | 2025

